In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

In [ ]:
# Save the size of train and test sets
ntrain = train.shape[0]
ntest = test.shape[0]

# Save the 'Id' column of train and test sets
train_ID = train['Id']
test_ID = test['Id']

# Drop the Id column which is irrevelant to our model building
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [ ]:
train.shape

In [ ]:
test.shape

## Data Explanation

**Data fields**

Here's a brief version of what you'll find in the data description file.

1. SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
1. MSSubClass: The building class
1. MSZoning: The general zoning classification
1. LotFrontage: Linear feet of street connected to property
1. LotArea: Lot size in square feet
1. Street: Type of road access
1. Alley: Type of alley access
1. LotShape: General shape of property
1. LandContour: Flatness of the property
1. Utilities: Type of utilities available
1. LotConfig: Lot configuration
1. LandSlope: Slope of property
1. Neighborhood: Physical locations within Ames city limits
1. Condition1: Proximity to main road or railroad
1. Condition2: Proximity to main road or railroad (if a second is present)
1. BldgType: Type of dwelling
1. HouseStyle: Style of dwelling
1. OverallQual: Overall material and finish quality
1. OverallCond: Overall condition rating
1. YearBuilt: Original construction date
1. YearRemodAdd: Remodel date
1. RoofStyle: Type of roof
1. RoofMatl: Roof material
1. Exterior1st: Exterior covering on house
1. Exterior2nd: Exterior covering on house (if more than one material)
1. MasVnrType: Masonry veneer type
1. MasVnrArea: Masonry veneer area in square feet
1. ExterQual: Exterior material quality
1. ExterCond: Present condition of the material on the exterior
1. Foundation: Type of foundation
1. BsmtQual: Height of the basement
1. BsmtCond: General condition of the basement
1. BsmtExposure: Walkout or garden level basement walls
1. BsmtFinType1: Quality of basement finished area
1. BsmtFinSF1: Type 1 finished square feet
1. BsmtFinType2: Quality of second finished area (if present)
1. BsmtFinSF2: Type 2 finished square feet
1. BsmtUnfSF: Unfinished square feet of basement area
1. TotalBsmtSF: Total square feet of basement area
1. Heating: Type of heating
1. HeatingQC: Heating quality and condition
1. CentralAir: Central air conditioning
1. Electrical: Electrical system
1. 1stFlrSF: First Floor square feet
1. 2ndFlrSF: Second floor square feet
1. LowQualFinSF: Low quality finished square feet (all floors)
1. GrLivArea: Above grade (ground) living area square feet
1. BsmtFullBath: Basement full bathrooms
1. BsmtHalfBath: Basement half bathrooms
1. FullBath: Full bathrooms above grade
1. HalfBath: Half baths above grade
1. Bedroom: Number of bedrooms above basement level
1. Kitchen: Number of kitchens
1. KitchenQual: Kitchen quality
1. TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
1. Functional: Home functionality rating
1. Fireplaces: Number of fireplaces
1. FireplaceQu: Fireplace quality
1. GarageType: Garage location
1. GarageYrBlt: Year garage was built
1. GarageFinish: Interior finish of the garage
1. GarageCars: Size of garage in car capacity
1. GarageArea: Size of garage in square feet
1. GarageQual: Garage quality
1. GarageCond: Garage condition
1. PavedDrive: Paved driveway
1. WoodDeckSF: Wood deck area in square feet
1. OpenPorchSF: Open porch area in square feet
1. EnclosedPorch: Enclosed porch area in square feet
1. 3SsnPorch: Three season porch area in square feet
1. ScreenPorch: Screen porch area in square feet
1. PoolArea: Pool area in square feet
1. PoolQC: Pool quality
1. Fence: Fence quality
1. MiscFeature: Miscellaneous feature not covered in other categories
1. MiscVal: $Value of miscellaneous feature
1. MoSold: Month Sold
1. YrSold: Year Sold
1. SaleType: Type of sale
1. SaleCondition: Condition of sale


## Data Preprocessing
- identify outliers 
- we are going to impute the missing values by the median (numerical) or mode (categorical) of the respective columns
- transform categorical variables to dummy variables

### 1. Outliers

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize = 13)
plt.xlabel('GrLivArea', fontsize = 13)
plt.show()

Note that the houses with more than 4000 square feet from the data set are outliers or unusual observations. It is suggested to remove them (eliminates 5 observations)

In [ ]:
# remove outliers and unusual observations
train = train.drop(train[train['GrLivArea'] > 4500].index)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize = 13)
plt.xlabel('GrLivArea', fontsize = 13)
plt.show()

## Target Variable 

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

sns.distplot(train['SalePrice'] , fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["SalePrice"] = np.log1p(train["SalePrice"])

#Check the new distribution 
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

In [ ]:
# Update the size of training set
ntrain = train.shape[0]

In [ ]:
# Combine training and testing sets into full data
y_train = train.SalePrice.values
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['SalePrice'], axis=1, inplace=True)

### 2. Data Imputation

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head()

In [ ]:
# Number of variables with missing data
missing_data.shape[0]

In [ ]:
# Variables correlation
plt.figure(figsize = (18,12))
corr = train.corr()
sns.heatmap(corr, annot=False)

In [ ]:
# 'NA' in PoolQC means there is no pool. We can replace it by 'None'
# 'NA' in MiscFeature means there is no feature, we can replace it by 'None'
# 'NA' in Alley means there is no alley access for the house, we can replace it by 'None'
# 'NA' in Fence means there is no fence for the house, we can replace it by 'None'
# 'NA' in FireplaceQu means there is no Fireplace, we can replace it by 'None'
for col in ('PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'):
    data[col] = data[col].fillna('None')
    
# LotFrontage:Since the area of each street connected to the house property most likely 
# have a similar area to other houses in its neighborhood , we can fill in missing values 
# by the median LotFrontage of the neighborhood. 
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

# 'NA' in GarageYrBlt implies there is no garage, we can replace it by 0. Similar for GarageArea 
# and GarageCars
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    data[col] = data[col].fillna(0)

# 'NA' in GarageFinish means there is no garage, this makes also the other related variables 
# GarageQual, GarageCond, GarageType becoming 'NA'. Replace all of them by 'None'
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    data[col] = data[col].fillna('None')

# Replace 'NA' in BsmtCond, BsmtExposure, BsmtQual, BsmtFinType2 and BsmtFinType1 by 'None'
for col in ('BsmtCond', 'BsmtExposure', 'BsmtQual', 'BsmtFinType2', 'BsmtFinType1'):
    data[col] = data[col].fillna('None')

# Replace 'NA' in MasVnrType by 'None' and replace 'NA' in MasVnrArea by 0
data['MasVnrType'] = data['MasVnrType'].fillna('None')
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)

# Replace 'NA' in MSZoning by the mode of it, which is 'RL'.
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])

# Replace 'NA' in BsmtFullBath, BsmtHalfBath, TotalBsmtSF, BsmtUnfSF, BsmtFinSF1, BsmtFinSF2 by 0 
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    data[col] = data[col].fillna(0)

# Replace 'NA' in Functional by the mode of it, that is 'Typ'
data['Functional'] = data['Functional'].fillna(data['Functional'].mode()[0])

# Replace 'NA' in Utilities by the mode of it, that is 'AllPub'
data['Utilities'] = data['Utilities'].fillna(data['Utilities'].mode()[0])

# Replace 'NA' in SaleType by the mode of it, that is 'WD'
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])

# Replace 'NA' in KitchenQual by the mode of it, that is 'TA'
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])

# Replace 'NA' in Electrical by the mode of it, that is 'TA'
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])

# Replace 'NA' in Electrical by the mode of it, that is 'TA'
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])


In [ ]:
np.sum(data.isnull())

### Incorrect Values

In [ ]:
data.describe()

In [ ]:
# The maximum value of GarageYrBlt is 2207 which is obviously wrong. It should be an data
# input error and the original value should be 2007. 

data[data['GarageYrBlt'] == 2207]

In [ ]:
data.loc[2590, 'GarageYrBlt'] = 2007

In [ ]:
data.info()

### Factorization 

In [ ]:
# Variable which should be object but it is read as numerical.
factors = ['MSSubClass']

for factor in factors:
    data.update(data[factor].astype('str'))

### Skewed features

In [ ]:
numeric_feats = data.dtypes[data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

### Box-Cox Transformation of highly skewed features

In [ ]:
skewness = skewness[abs(skewness.Skew) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    data[feat] = boxcox1p(data[feat], lam)

### Check if the level of features are too low which may require deletion

In [ ]:
objects = []
for i in data.columns:
    if data[i].dtype == object:
        objects.append(i)

In [ ]:
sum_features = data[objects].apply(lambda x: len(np.unique(x)))
sum_features.sort_values(ascending = False)

In [ ]:
print(data['Utilities'].value_counts())
print('------')
print(data['Street'].value_counts())
print('------')
print(data['CentralAir'].value_counts())
print('------')
print(data['PavedDrive'].value_counts())
print('------')

In [ ]:
# Since Utilites and Street has low amount of level and most of the data are in 
# the same class(>99%), we decide to delete them

data = data.drop(['Utilities', 'Street'], axis=1)

### Create features

In [ ]:
data['Total_sqr_footage'] = (data['BsmtFinSF1'] + data['BsmtFinSF2'] +
                                 data['1stFlrSF'] + data['2ndFlrSF'])

data['Total_Bathrooms'] = (data['FullBath'] + (0.5*data['HalfBath']) + 
                               data['BsmtFullBath'] + (0.5*data['BsmtHalfBath']))

data['Total_porch_sf'] = (data['OpenPorchSF'] + data['3SsnPorch'] +
                              data['EnclosedPorch'] + data['ScreenPorch'] +
                             data['WoodDeckSF'])


#simplified features
data['haspool'] = data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
data['has2ndfloor'] = data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
data['hasgarage'] = data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
data['hasbsmt'] = data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
data['hasfireplace'] = data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

### 3. Transform categorical features to dummy variables

In [ ]:
data.shape

In [ ]:
# create dummy variables for categorical features
data = pd.get_dummies(data).reset_index(drop=True)

In [ ]:
# overall data shape
data.shape

## Overfitting preventation

In [ ]:
X = data.iloc[:ntrain,:]

In [ ]:
X['haspool'].value_counts().iloc[0]/len(X)

In [ ]:
# To prevent overfitting, dummy columns with more than 97% 1 or 0 will be removed.

overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.94:
        overfit.append(i)

In [ ]:
overfit

In [ ]:
data.drop(overfit, axis=1, inplace=True)
data.shape

In [ ]:
# resplit the data into training and testing sets
train = data[:ntrain]
test = data[ntrain:]

# Building Model

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
# Define the covariate matrix for model training and prediction
X_train = train
X_test = test

In [ ]:
#Validation function
n_folds = 10

def rmsle_cv(model):
    kfold = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kfold))
    return(rmse)

## Lasso regression

In [ ]:
# This model may be very sensitive to outliers. 
# So we need to made it more robust on them. For that we use the sklearn's Robustscaler() method on pipeline
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=22))

## Elastic Net Regression

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=10))

## Kernel Ridge Regression

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

## Gradient Boosting Regression

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

## XGBoost

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

## LightGBM

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

## Random Forest Regression model

In [ ]:
# Grid Search
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV

#regressor = RandomForestRegressor()
#parameters = [{'n_estimators' : [100,150,200,250,300], 'max_features' : ['auto','sqrt','log2']}]
#grid_search = GridSearchCV(estimator = regressor, param_grid = parameters)
#grid_search = grid_search.fit(X_train, y_train)
#best_parameters = grid_search.best_params_
#best_accuracy = grid_search.best_score_

In [ ]:
#best_parameters

In [ ]:
# Random Forest Regression model
# Use the best parameters found from above to build the model

#RF = RandomForestRegressor(n_estimators = 300, max_features = 'auto') 


## Base Model score

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("Elastic Net score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#score = rmsle_cv(RF)
#print("Random Forest score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("XGBoost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LightGBM score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

## Stacking models

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [ ]:
averaged_models = AveragingModels(models = (lasso, ENet, GBoost, model_xgb))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

## Less simple Stacking : Adding a Meta-model

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, model_xgb),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Stacked model

In [ ]:
stacked_averaged_models.fit(X_train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(X_train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(X_test.values))
print(rmsle(y_train, stacked_train_pred))

### LightGBM 

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 + lgb_train_pred*0.3 ))

### Ensemble prediction

In [ ]:
ensemble = stacked_pred*0.7 + lgb_pred*0.3

## Submission

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)